In [6]:
import os
import zipfile
import random
import shutil
import numpy as np
from shutil import copyfile
import tensorflow as tf
import matplotlib.pyplot as plt
from PIL import Image
from tensorflow.keras.preprocessing.image import ImageDataGenerator
import string
from tensorflow.keras.applications.mobilenet_v2 import MobileNetV2

In [7]:
source_path = 'D:\Reza MZ\yosa-capstone\data'
new_root = 'D:\Reza MZ\yosa-capstone\splited_data'

classes =  os.listdir(source_path)

for cls in classes:
    os.makedirs(os.path.join(new_root, "training", cls))
    os.makedirs(os.path.join(new_root, "testing", cls))

FileExistsError: [WinError 183] Cannot create a file when that file already exists: 'D:\\Reza MZ\\yosa-capstone\\splited_data\\training\\badhakonasana'

In [8]:
for rootdir, dirs, files in os.walk(new_root):
    for subdir in dirs:
        print(os.path.join(rootdir, subdir))

D:\Reza MZ\yosa-capstone\splited_data\testing
D:\Reza MZ\yosa-capstone\splited_data\training
D:\Reza MZ\yosa-capstone\splited_data\testing\badhakonasana
D:\Reza MZ\yosa-capstone\splited_data\testing\cat_and_cow
D:\Reza MZ\yosa-capstone\splited_data\testing\chair
D:\Reza MZ\yosa-capstone\splited_data\testing\chaturanga
D:\Reza MZ\yosa-capstone\splited_data\testing\child
D:\Reza MZ\yosa-capstone\splited_data\testing\downward_dog
D:\Reza MZ\yosa-capstone\splited_data\testing\high_plank
D:\Reza MZ\yosa-capstone\splited_data\testing\mountain
D:\Reza MZ\yosa-capstone\splited_data\testing\mountain_with_arms_up
D:\Reza MZ\yosa-capstone\splited_data\testing\mountain_with_prayer_hand
D:\Reza MZ\yosa-capstone\splited_data\testing\one_legged_table
D:\Reza MZ\yosa-capstone\splited_data\testing\parivrtta_sukhasana
D:\Reza MZ\yosa-capstone\splited_data\testing\plank_with_knee_to_tricep
D:\Reza MZ\yosa-capstone\splited_data\testing\shavasana
D:\Reza MZ\yosa-capstone\splited_data\testing\standing_forwa

In [9]:
def split_data(SOURCE, TRAINING, TESTING, SPLIT_SIZE):

  files = []
  for filename in os.listdir(SOURCE):
    file = os.path.join(SOURCE, filename)
    if os.path.getsize(file) > 0:
      files.append(filename)
    else:
      print("{} is zero length, so ignoring.".format(filename))
    
  training_size = int(SPLIT_SIZE)
  testing_size = int(len(files) - training_size)
  random_list = random.sample(files, len(files))
  training_list = files[:training_size]
  testing_list = files[-testing_size:]

  for filename in training_list:
    copyfile(os.path.join(SOURCE, filename), os.path.join(TRAINING, filename))

  for filename in testing_list:
    copyfile(os.path.join(SOURCE, filename), os.path.join(TESTING, filename))

  pass

In [10]:
training_dir = os.path.join(new_root, 'training')
testing_dir = os.path.join(new_root, 'testing')
for cls in classes:
    split_data(os.path.join(source_path, cls), os.path.join(training_dir, cls), os.path.join(testing_dir, cls), 20)

c = 1
for folder in os.listdir("D:\Reza MZ\yosa-capstone\splited_data"):
    for cls in os.listdir(os.path.join("D:\Reza MZ\yosa-capstone\splited_data", folder)):
        for filename in os.listdir(os.path.join("D:\Reza MZ\yosa-capstone\splited_data", folder, cls)):
            if filename.endswith(".jpg"):
                im = Image.open(os.path.join("D:\Reza MZ\yosa-capstone\splited_data", folder, cls, filename))
                name = 'img'+str(c)+'.png'
                rgb_im = im.convert('RGB')
                rgb_im.save(os.path.join("D:\Reza MZ\yosa-capstone\splited_data", folder, cls, name))
                os.remove(os.path.join("D:\Reza MZ\yosa-capstone\splited_data", folder, cls, filename))
                c += 1
                continue
            else:
                continue

In [11]:
def train_val_generators(TRAINING_DIR, VALIDATION_DIR):
    training_datagen = ImageDataGenerator(
          rescale = 1./255,
          horizontal_flip=True,
          fill_mode='nearest')

    validation_datagen = ImageDataGenerator(rescale = 1./255)

    train_generator = training_datagen.flow_from_directory(
        TRAINING_DIR,
        target_size=(224,224),
        class_mode='categorical',
        batch_size=16
    )

    validation_generator = validation_datagen.flow_from_directory(
        VALIDATION_DIR,
        target_size=(224,224),
        class_mode='categorical',
        batch_size=16
    )
    return train_generator, validation_generator

In [18]:
train_generator, validation_generator = train_val_generators(training_dir, testing_dir)

Found 360 images belonging to 18 classes.
Found 180 images belonging to 18 classes.


In [8]:
pre_trained_model = MobileNetV2(input_shape=(224, 224, 3),
                               include_top=False)
for layer in pre_trained_model.layers:
    layer.trainable = False

In [9]:
image_batch, label_batch = next(iter(train_generator))
feature_batch = pre_trained_model(image_batch)
print(feature_batch.shape)

(16, 7, 7, 1280)


In [10]:
pre_trained_model.summary()

total_params = pre_trained_model.count_params()
num_trainable_params = sum([w.shape.num_elements() for w in pre_trained_model.trainable_weights])

print(f"There are {total_params:,} total parameters in this model.")
print(f"There are {num_trainable_params:,} trainable parameters in this model.")

Model: "mobilenetv2_1.00_224"
__________________________________________________________________________________________________
 Layer (type)                   Output Shape         Param #     Connected to                     
 input_1 (InputLayer)           [(None, 224, 224, 3  0           []                               
                                )]                                                                
                                                                                                  
 Conv1 (Conv2D)                 (None, 112, 112, 32  864         ['input_1[0][0]']                
                                )                                                                 
                                                                                                  
 bn_Conv1 (BatchNormalization)  (None, 112, 112, 32  128         ['Conv1[0][0]']                  
                                )                                              

 block_3_expand_relu (ReLU)     (None, 56, 56, 144)  0           ['block_3_expand_BN[0][0]']      
                                                                                                  
 block_3_pad (ZeroPadding2D)    (None, 57, 57, 144)  0           ['block_3_expand_relu[0][0]']    
                                                                                                  
 block_3_depthwise (DepthwiseCo  (None, 28, 28, 144)  1296       ['block_3_pad[0][0]']            
 nv2D)                                                                                            
                                                                                                  
 block_3_depthwise_BN (BatchNor  (None, 28, 28, 144)  576        ['block_3_depthwise[0][0]']      
 malization)                                                                                      
                                                                                                  
 block_3_d

 lization)                                                                                        
                                                                                                  
 block_7_expand (Conv2D)        (None, 14, 14, 384)  24576       ['block_6_project_BN[0][0]']     
                                                                                                  
 block_7_expand_BN (BatchNormal  (None, 14, 14, 384)  1536       ['block_7_expand[0][0]']         
 ization)                                                                                         
                                                                                                  
 block_7_expand_relu (ReLU)     (None, 14, 14, 384)  0           ['block_7_expand_BN[0][0]']      
                                                                                                  
 block_7_depthwise (DepthwiseCo  (None, 14, 14, 384)  3456       ['block_7_expand_relu[0][0]']    
 nv2D)    

                                                                                                  
 block_10_depthwise_relu (ReLU)  (None, 14, 14, 384)  0          ['block_10_depthwise_BN[0][0]']  
                                                                                                  
 block_10_project (Conv2D)      (None, 14, 14, 96)   36864       ['block_10_depthwise_relu[0][0]']
                                                                                                  
 block_10_project_BN (BatchNorm  (None, 14, 14, 96)  384         ['block_10_project[0][0]']       
 alization)                                                                                       
                                                                                                  
 block_11_expand (Conv2D)       (None, 14, 14, 576)  55296       ['block_10_project_BN[0][0]']    
                                                                                                  
 block_11_

 block_14_depthwise (DepthwiseC  (None, 7, 7, 960)   8640        ['block_14_expand_relu[0][0]']   
 onv2D)                                                                                           
                                                                                                  
 block_14_depthwise_BN (BatchNo  (None, 7, 7, 960)   3840        ['block_14_depthwise[0][0]']     
 rmalization)                                                                                     
                                                                                                  
 block_14_depthwise_relu (ReLU)  (None, 7, 7, 960)   0           ['block_14_depthwise_BN[0][0]']  
                                                                                                  
 block_14_project (Conv2D)      (None, 7, 7, 160)    153600      ['block_14_depthwise_relu[0][0]']
                                                                                                  
 block_14_

In [11]:
global_average_layer = tf.keras.layers.GlobalAveragePooling2D()
feature_batch_average = global_average_layer(feature_batch)
print(feature_batch_average.shape)

(16, 1280)


In [12]:
prediction_layer = tf.keras.layers.Dense(18, activation='softmax')
prediction_batch = prediction_layer(feature_batch_average)
print(prediction_batch.shape)

(16, 18)


In [13]:
data_augmentation = tf.keras.Sequential([
  tf.keras.layers.RandomFlip('horizontal'),
])
inputs = tf.keras.Input(shape=(224, 224, 3))
x = data_augmentation(inputs)
x = tf.keras.applications.mobilenet_v2.preprocess_input(x)
x = pre_trained_model(x, training=False)
x = global_average_layer(x)
x = tf.keras.layers.Dropout(0.2)(x)
outputs = prediction_layer(x)
model = tf.keras.Model(inputs, outputs)

In [14]:
model.compile(loss = 'categorical_crossentropy', optimizer='adam', metrics=['accuracy'])

In [15]:
history = model.fit(train_generator,
                    epochs=600,
                    verbose=1,
                    validation_data=validation_generator)

Epoch 1/600
23/23 [==============================] - 22s 798ms/step - loss: 3.2045 - accuracy: 0.0306 - val_loss: 2.9392 - val_accuracy: 0.0667
Epoch 2/600
23/23 [==============================] - 15s 679ms/step - loss: 3.0469 - accuracy: 0.0417 - val_loss: 2.9082 - val_accuracy: 0.0944
Epoch 3/600
23/23 [==============================] - 23s 1s/step - loss: 3.0384 - accuracy: 0.0528 - val_loss: 2.9086 - val_accuracy: 0.0556
Epoch 4/600
23/23 [==============================] - 17s 724ms/step - loss: 3.0070 - accuracy: 0.0639 - val_loss: 2.8937 - val_accuracy: 0.0556
Epoch 5/600
23/23 [==============================] - 21s 841ms/step - loss: 3.0458 - accuracy: 0.0528 - val_loss: 2.8986 - val_accuracy: 0.0667
Epoch 6/600
23/23 [==============================] - 20s 881ms/step - loss: 2.9800 - accuracy: 0.0583 - val_loss: 2.9151 - val_accuracy: 0.0611
Epoch 7/600
23/23 [==============================] - 20s 887ms/step - loss: 2.9803 - accuracy: 0.0611 - val_loss: 2.8830 - val_accuracy: 0.

23/23 [==============================] - 16s 707ms/step - loss: 2.3880 - accuracy: 0.2472 - val_loss: 2.4417 - val_accuracy: 0.2444
Epoch 114/600
23/23 [==============================] - 16s 717ms/step - loss: 2.3674 - accuracy: 0.3083 - val_loss: 2.4418 - val_accuracy: 0.2000
Epoch 115/600
23/23 [==============================] - 16s 712ms/step - loss: 2.3581 - accuracy: 0.2583 - val_loss: 2.4208 - val_accuracy: 0.2778
Epoch 116/600
23/23 [==============================] - 16s 711ms/step - loss: 2.3636 - accuracy: 0.2889 - val_loss: 2.4566 - val_accuracy: 0.2333
Epoch 117/600
23/23 [==============================] - 16s 719ms/step - loss: 2.3835 - accuracy: 0.2917 - val_loss: 2.4174 - val_accuracy: 0.2722
Epoch 118/600
23/23 [==============================] - 16s 714ms/step - loss: 2.3708 - accuracy: 0.2778 - val_loss: 2.4403 - val_accuracy: 0.2500
Epoch 119/600
23/23 [==============================] - 16s 716ms/step - loss: 2.3331 - accuracy: 0.3000 - val_loss: 2.4203 - val_accuracy:

23/23 [==============================] - 16s 719ms/step - loss: 2.1517 - accuracy: 0.3361 - val_loss: 2.2475 - val_accuracy: 0.3000
Epoch 226/600
23/23 [==============================] - 17s 728ms/step - loss: 2.0915 - accuracy: 0.3306 - val_loss: 2.2420 - val_accuracy: 0.3611
Epoch 227/600
23/23 [==============================] - 16s 708ms/step - loss: 2.1410 - accuracy: 0.3333 - val_loss: 2.2267 - val_accuracy: 0.3056
Epoch 228/600
23/23 [==============================] - 16s 707ms/step - loss: 2.1651 - accuracy: 0.3083 - val_loss: 2.2278 - val_accuracy: 0.3333
Epoch 229/600
23/23 [==============================] - 17s 733ms/step - loss: 2.1030 - accuracy: 0.3389 - val_loss: 2.2647 - val_accuracy: 0.3111
Epoch 230/600
23/23 [==============================] - 16s 720ms/step - loss: 2.1670 - accuracy: 0.3056 - val_loss: 2.2332 - val_accuracy: 0.3333
Epoch 231/600
23/23 [==============================] - 16s 710ms/step - loss: 2.1054 - accuracy: 0.4000 - val_loss: 2.2131 - val_accuracy:

23/23 [==============================] - 16s 720ms/step - loss: 2.0235 - accuracy: 0.3667 - val_loss: 2.1421 - val_accuracy: 0.3722
Epoch 338/600
23/23 [==============================] - 17s 721ms/step - loss: 1.9587 - accuracy: 0.4056 - val_loss: 2.1339 - val_accuracy: 0.3111
Epoch 339/600
23/23 [==============================] - 16s 717ms/step - loss: 1.9440 - accuracy: 0.4056 - val_loss: 2.1478 - val_accuracy: 0.3333
Epoch 340/600
23/23 [==============================] - 16s 722ms/step - loss: 2.0066 - accuracy: 0.4056 - val_loss: 2.1191 - val_accuracy: 0.3444
Epoch 341/600
23/23 [==============================] - 16s 722ms/step - loss: 1.9533 - accuracy: 0.4250 - val_loss: 2.1199 - val_accuracy: 0.3500
Epoch 342/600
23/23 [==============================] - 16s 721ms/step - loss: 1.9565 - accuracy: 0.4000 - val_loss: 2.1085 - val_accuracy: 0.3833
Epoch 343/600
23/23 [==============================] - 16s 713ms/step - loss: 1.9775 - accuracy: 0.3917 - val_loss: 2.1122 - val_accuracy:

23/23 [==============================] - 16s 707ms/step - loss: 1.8736 - accuracy: 0.4028 - val_loss: 2.0517 - val_accuracy: 0.3889
Epoch 450/600
23/23 [==============================] - 16s 707ms/step - loss: 1.9153 - accuracy: 0.4167 - val_loss: 2.0430 - val_accuracy: 0.3722
Epoch 451/600
23/23 [==============================] - 16s 712ms/step - loss: 1.8046 - accuracy: 0.4639 - val_loss: 2.0290 - val_accuracy: 0.3556
Epoch 452/600
23/23 [==============================] - 16s 710ms/step - loss: 1.8384 - accuracy: 0.4444 - val_loss: 2.0650 - val_accuracy: 0.3389
Epoch 453/600
23/23 [==============================] - 16s 708ms/step - loss: 1.8264 - accuracy: 0.4361 - val_loss: 2.0295 - val_accuracy: 0.3611
Epoch 454/600
23/23 [==============================] - 17s 723ms/step - loss: 1.9001 - accuracy: 0.4139 - val_loss: 2.0430 - val_accuracy: 0.3444
Epoch 455/600
23/23 [==============================] - 16s 717ms/step - loss: 1.8277 - accuracy: 0.4389 - val_loss: 2.0213 - val_accuracy:

23/23 [==============================] - 17s 753ms/step - loss: 1.7488 - accuracy: 0.4944 - val_loss: 1.9545 - val_accuracy: 0.3722
Epoch 562/600
23/23 [==============================] - 17s 731ms/step - loss: 1.7692 - accuracy: 0.4861 - val_loss: 1.9728 - val_accuracy: 0.4056
Epoch 563/600
23/23 [==============================] - 16s 737ms/step - loss: 1.7395 - accuracy: 0.4528 - val_loss: 1.9775 - val_accuracy: 0.4389
Epoch 564/600
23/23 [==============================] - 17s 730ms/step - loss: 1.7896 - accuracy: 0.4639 - val_loss: 1.9933 - val_accuracy: 0.3944
Epoch 565/600
23/23 [==============================] - 17s 726ms/step - loss: 1.7829 - accuracy: 0.4528 - val_loss: 1.9791 - val_accuracy: 0.3667
Epoch 566/600
23/23 [==============================] - 17s 733ms/step - loss: 1.7565 - accuracy: 0.4361 - val_loss: 1.9843 - val_accuracy: 0.3444
Epoch 567/600
23/23 [==============================] - 17s 732ms/step - loss: 1.7605 - accuracy: 0.4583 - val_loss: 2.0038 - val_accuracy:

In [16]:
export_dir = 'D:\Reza MZ\yosa-capstone'
tf.saved_model.save(model, export_dir)

INFO:tensorflow:Assets written to: D:\Reza MZ\yosa-capstone\assets


In [17]:
converter = tf.lite.TFLiteConverter.from_saved_model(export_dir)
tflite_model = converter.convert()

In [21]:
import pathlib
tflite_model_file = pathlib.Path('D:\Reza MZ\yosa-capstone\poses.tflite') 
tflite_model_file.write_bytes(tflite_model)

8959584

In [3]:
model = tf.saved_model.load('D:\Reza MZ\yosa-capstone')

In [19]:
model.labels

AttributeError: '_UserObject' object has no attribute 'labels'